In [3]:
"""
파일: raptor_rag_search_example.py

목적:
- RAPTOR 인덱스(요약+원문이 함께 들어있는 FAISS)를 로드하고
- 2단계 검색(요약 코스 검색 → L0 파인 검색)을 수행
- (선택) BM25와 RRF 결합
- 결과 출력

의존:
- langchain_community, langchain_core
- sentence-transformers (HuggingFaceEmbeddings를 위해, 이미 인덱스 생성 때 사용)
"""

from typing import List, Dict, Any, Iterable
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document

# =========================
# 유틸: E5/BGE 프리픽스
# =========================
def qtext(q: str) -> str:
    """E5/BGE 계열은 쿼리에 'query: ' 프리픽스를 붙이는 것이 성능에 유리"""
    return f"query: {q}"

def is_summary(doc: Document) -> bool:
    """요약 문서 판정: level 메타데이터(1,2,...)가 붙은 문서"""
    return "level" in doc.metadata and int(doc.metadata["level"]) >= 1

def is_leaf(doc: Document) -> bool:
    """원문(L0) 판정: level==0 또는 level 키 자체가 없는 문서(과거 인덱스 호환)"""
    lvl = doc.metadata.get("level", None)
    return lvl == 0 or lvl is None

# =========================
# 1) 요약 코스 검색
# =========================
def coarse_search_summaries(vs: FAISS, query: str, k: int = 6) -> List[Document]:
    """
    level>=1 요약문만 대상으로 상위 k개를 고름.
    (FAISS 기본 필터가 단순하여, 넉넉히 가져온 후 파이썬에서 요약만 걸러냄)
    """
    candidates = vs.similarity_search(qtext(query), k=40)  # 넉넉히
    summaries = [d for d in candidates if is_summary(d)]
    return summaries[:k]

# =========================
# 2) 요약 → 소스 스코프 축소
# =========================
def collect_source_paths(summaries: Iterable[Document]) -> List[str]:
    """
    요약 메타데이터의 source_files에서 파일 경로 목록을 수집.
    """
    paths = set()
    for d in summaries:
        srcs = d.metadata.get("source_files", [])
        if isinstance(srcs, str):
            paths.add(srcs)
        else:
            paths.update(srcs)
    # 요약에 source_files가 비어있다면, 문서별 'source' 키를 병합해도 됨.
    return sorted(p for p in paths if p)

# =========================
# 3) 파인 검색 (L0만, 스코프 내)
# =========================
def fine_search_leaves_in_scope(vs: FAISS, query: str, source_paths: List[str], k: int = 8) -> List[Document]:
    """
    수집한 파일 경로(source_paths) 범위 내에서 L0만 재검색.
    간단히 많이 가져와서(in-scope)만 필터링.
    """
    candidates = vs.similarity_search(qtext(query), k=120)  # 넉넉히
    in_scope: List[Document] = []
    for d in candidates:
        if not is_leaf(d):
            continue
        src = d.metadata.get("source") or d.metadata.get("path") or ""
        if not src:
            continue
        # 경로가 완전일치 or 부분 포함일치면 스코프에 포함
        if any((sp == src) or (sp in src) for sp in source_paths):
            in_scope.append(d)

    return in_scope[:k]

# =========================
# (선택) 4) RRF 결합 (Dense + BM25)
# =========================
def rrf_merge(dense_docs: List[Document], sparse_docs: List[Document], k: int = 8, C: int = 60) -> List[Document]:
    """
    Reciprocal Rank Fusion(간단 구현)
    두 랭킹 리스트를 합쳐 상위 k개만 반환
    """
    from collections import defaultdict
    score = defaultdict(float)

    # helper: 문서를 (id)로 식별
    def add_scores(ranked: List[Document]):
        for rank, d in enumerate(ranked, start=1):
            score[id(d)] += 1.0 / (C + rank)

    add_scores(dense_docs)
    add_scores(sparse_docs)

    # 순서는 dense_docs를 기준으로 시작, 없는 항목은 뒤에 추가
    order = list(dense_docs) + [d for d in sparse_docs if d not in dense_docs]
    order.sort(key=lambda d: score[id(d)], reverse=True)
    return order[:k]

# =========================
# 5) 최종 검색 파이프라인
# =========================
def raptor_search(
    vs_dense: FAISS,
    query: str,
    k_final: int = 6,
    bm25_index=None  # 선택: 스파스 인덱스(예: rank_bm25 또는 ES)
) -> Dict[str, Any]:
    """
    2단계 검색:
    - (코스) 요약만 검색 → 관련 클러스터 선택
    - (파인) 해당 클러스터의 source_files 범위에서 L0 재검색
    - (선택) BM25 결합
    반환: {'summaries': [...], 'leaves': [...], 'final': [...]}
    """
    # 1) 요약 코스 검색
    summaries = coarse_search_summaries(vs_dense, query, k=6)

    # 2) 요약 → 소스 파일 스코프 축소
    scope_paths = collect_source_paths(summaries)

    # 3) 파인 검색(L0, 스코프 내)
    leaves_dense = fine_search_leaves_in_scope(vs_dense, query, scope_paths, k=16)

    # 4) BM25(선택) 결합
    if bm25_index is not None:
        # bm25_index.search는 프로젝트 구현체에 맞게 바꿔주세요.
        sparse_hits: List[Document] = bm25_index.search(query, k=20)
        final_docs = rrf_merge(leaves_dense, sparse_hits, k=k_final)
    else:
        final_docs = leaves_dense[:k_final]

    return {"summaries": summaries, "leaves": leaves_dense, "final": final_docs}

# =========================
# (옵션) 벡터스토어 로드
# =========================
def load_vector_store(path: str, embedding_model_name: str = "intfloat/multilingual-e5-base") -> FAISS:
    """
    인덱싱 때 썼던 임베딩 모델과 '동일한' 모델로 로드해야 합니다.
    (임베딩 모델이 다르면 검색 품질이 무너집니다.)
    """
    embedder = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        model_kwargs={"device": "cuda"},       # GPU 없으면 "cpu"
        encode_kwargs={"normalize_embeddings": True},
    )
    vs = FAISS.load_local(path, embeddings=embedder, allow_dangerous_deserialization=True)
    return vs

# =========================
# (옵션) 출력 헬퍼
# =========================
def print_hits(title: str, docs: List[Document], max_chars: int = 200):
    print(f"\n=== {title} (n={len(docs)}) ===")
    for i, d in enumerate(docs, 1):
        txt = d.page_content.replace("\n", " ")
        if len(txt) > max_chars:
            txt = txt[:max_chars] + " ..."
        print(f"[{i}] {txt}")
        print("    meta:", d.metadata)

# =========================
# 메인: 사용 예시
# =========================
if __name__ == "__main__":
    # 0) 벡터스토어 로드 (경로는 환경에 맞게)
    VS_PATH = "/data1/home/gmk/SL/client/vector_stores/my_project_raptor_db"
    vs = load_vector_store(VS_PATH, embedding_model_name="intfloat/multilingual-e5-base")

    # 1) 사용자 질의 (한국어 본문이면 한국어 질의가 유리)
    user_query = "자동차 연료 상태를 확인하는 함수"

    # 2) RAPTOR 2-단계 검색 실행
    result = raptor_search(vs, user_query, k_final=5, bm25_index=None)  # bm25 사용 시 객체 전달

    # 3) 결과 확인
    print_hits("요약(코스 검색 결과)", result["summaries"])
    print_hits("원문 L0(스코프 내 파인 검색 결과)", result["leaves"])
    print_hits("최종 선택(결합/재순위화 이후)", result["final"])

    # 이후: result["final"]의 L0 스니펫만 LLM에 넘겨 답변 생성 시 “제공된 스니펫에서만 근거 사용” 규칙을 적용하세요.



=== 요약(코스 검색 결과) (n=5) ===
[1] 언어: C 핵심 요약: 자동차 상태를 초기화, 시동 켜기, 끄기, 기어 변경, 상태 표시, 주유, 제동을 위한 함수들이 제공된다.   주요 구성요소: - 구조체 `Car`(필드: `brand`, `model`, `speed`, `max_speed`, `fuel`, `odometer`, `gear`, `engine_on`) - 함수 `init_car(Car* ...
    meta: {'source_files': ['/data1/home/gmk/SL/client/test/car.c'], 'level': 1}
[2] 언어: C  핵심 요약: 이 코드는 차량 시뮬레이션을 위한 C 프로그램으로, 차량의 속도, 연료, 기어, 시동 상태를 관리합니다. 프로그램은 시뮬레이션 틱 업데이트, 가속, 브레이크, 리필, 시동 켜기/끄기, 기어 변경, 상태 표시 등 다양한 기능을 제공합니다.  주요 구성요소: - 구조체 `Car`(필드: `brand`, `model`, `speed`,  ...
    meta: {'source_files': ['/data1/home/gmk/SL/client/test/car.c'], 'level': 2}
[3] 언어: C 핵심 요약: 이 코드는 차량 시뮬레이션을 위한 C 프로그램으로, 차량의 속도, 연료, 기어, 시동 상태를 관리합니다. 프로그램은 시뮬레이션 틱 업데이트, 가속, 브레이크, 리필, 시동 켜기/끄기, 기어 변경, 상태 표시 등 다양한 기능을 제공합니다.  주요 구성요소: - 구조체 `Car`(필드: `brand`, `model`, `speed`, ` ...
    meta: {'source_files': ['/data1/home/gmk/SL/client/test/car.c'], 'level': 1}
[4] 언어: C  핵심 요약: 이 코드는 기어 상태를 나타내는 열거형 `enum`과 `gear_to_str` 함수를 정의합니다. 이 함수는 열거형 `Gear`의 값을 문자열로 변환합